In [2]:
import pandas as pd

In [36]:
# Import data
snapshot_1 = pd.read_csv('data/snapshot_1.csv')
snapshot_2 = pd.read_csv('data/snapshot_2.csv')

# rename snapshot_2's columns to match snapshot_1's assuming they have the same column data in the same column order
snapshot_2.columns = snapshot_1.columns

In [37]:
# cleans the sku's to have uppercase letters for the 'SKU' part of their item code
snapshot_2['sku'] = snapshot_2['sku'].str.upper()
# adds a dash between the first capture group ('SKU') and second capture group (###)
snapshot_2['sku'] = snapshot_2['sku'].str.replace(r'(SKU)(\d+)', r'\1-\2', regex=True)

In [38]:
# check for duplicated sku's or names
print(
    snapshot_1['sku'].is_unique,
    snapshot_2['sku'].is_unique,
    snapshot_1['name'].is_unique,
    snapshot_2['name'].is_unique
)

True False True True


In [39]:
# Strip whitespace from item names
snapshot_1['name'] = snapshot_1['name'].str.strip()
snapshot_2['name'] = snapshot_2['name'].str.strip()

# Found through
# snapshot_1['name'].values
# snapshot_2['name'].values

In [42]:
# Create slice to find duplicated sku
df_duplicates = snapshot_2[snapshot_2.duplicated(subset='sku', keep=False)]
df_duplicates

,sku,name,quantity,location,last_counted


In [41]:
# groups and merges duplicated sku's
agg_rules = {
    'sku': 'first', # takes first value that appears
    'name': 'last', # went with second record to match first snapshot - unsure what official name is
    'quantity': 'sum', # adds the two values together
    'location': 'first',
    'last_counted': 'first'
}

snapshot_2 = snapshot_2.groupby('sku', as_index=False).agg(agg_rules)

In [43]:
# Compare sku names across snapshots
comparison_df = snapshot_1[['sku', 'name']].merge(
    snapshot_2[['sku', 'name']],
    on='sku',
    suffixes=('_snap1', '_snap2')
)

# Create a 'match' column to easily filter
comparison_df['is_match'] = comparison_df['name_snap1'] == comparison_df['name_snap2']

# View only the mismatches
mismatches = comparison_df[comparison_df['is_match'] == False]
mismatches

,sku,name_snap1,name_snap2,is_match


In [49]:
# Normalize data type for quantity between snapshots
snapshot_2['quantity'] = snapshot_2['quantity'].astype(snapshot_1['quantity'].dtype)

# found through
# snapshot_1['quantity'].values
# snapshot_2['quantity'].values
# snapshot_1['quantity'].dtype
# snapshot_2['quantity'].dtype

In [51]:
# Check location values
snapshot_1['location'].value_counts()

location
Warehouse A    33
Warehouse B    23
Warehouse C    19
Name: count, dtype: int64

In [52]:
# Check location values
snapshot_2['location'].value_counts()

location
Warehouse A    38
Warehouse B    21
Warehouse C    19
Name: count, dtype: int64

In [61]:
# identified issue with snapshot_2 last_counted formatting
# snapshot_1['last_counted'].value_counts()
snapshot_2['last_counted'].value_counts()

last_counted
2024-01-15    77
01/15/2024     1
Name: count, dtype: int64

In [63]:
# Converted column values to datetime before normalizing to a single format
snapshot_2['last_counted'] = pd.to_datetime(snapshot_2['last_counted'], format='mixed')
snapshot_2['last_counted'] = snapshot_2['last_counted'].dt.strftime('%Y-%m-%d')
snapshot_2['last_counted'].value_counts()

last_counted
2024-01-15    78
Name: count, dtype: int64

In [64]:
# examining skus for each reconciliation status
new_skus = [sku for sku in snapshot_2['sku'].values if sku not in snapshot_1['sku'].values]
removed_skus = [sku for sku in snapshot_1['sku'].values if sku not in snapshot_2['sku'].values]
retained_skus = [sku for sku in snapshot_1['sku'].values if sku in snapshot_2['sku'].values]

len(retained_skus), len(new_skus), len(removed_skus)

(73, 5, 2)

In [65]:
removed_skus

['SKU-025', 'SKU-026']

In [68]:
def reconciliation_analysis(snapshot_old, snapshot_new):
    # Merge snapshots on 'sku' while retaining both old and new values
    merged = pd.merge(snapshot_old, snapshot_new, on='sku', how='outer', indicator='status', suffixes=('_old', '_new'))

    # Add a status based on the type of merge to differentiate SKU retention
    status_map = {
        'left_only': 'Removed',
        'right_only': 'New',
        'both': 'Retained'
    }

    merged['status'] = merged['status'].map(status_map)

    return merged

In [69]:
reconciliation_df = reconciliation_analysis(snapshot_1, snapshot_2)

In [78]:
# examining newly merged df from function
# reconciliation_df
# reconciliation_df[reconciliation_df['status'] != 'Retained']

In [79]:
# Create new df's that separate sku's based on status
# dropna removes empty columns for slices
new_df = reconciliation_df[reconciliation_df['status'] == 'New'].dropna(axis=1, how='all').copy()
removed_df = reconciliation_df[reconciliation_df['status'] == 'Removed'].dropna(axis=1, how='all').copy()
retained_df = reconciliation_df[reconciliation_df['status'] == 'Retained'].copy()

In [83]:
# examining new df's to check removed columns
# new_df
# removed_df
# retained_df

In [85]:
# created 'inventory_change' column
retained_df['inventory_change'] = retained_df['quantity_new'] - retained_df['quantity_old']
# retained_df

In [87]:
# Removed 'status' column for each df
new_df.drop(columns=['status'], inplace=True)
removed_df.drop(columns=['status'], inplace=True)
retained_df.drop(columns=['status', 'name_new'], inplace=True)

In [102]:
# new_df
# removed_df
# retained_df

In [91]:
# Rename columns in new_df to remove suffixes
new_df.rename(columns={
    'name_new': 'name',
    'quantity_new': 'quantity',
    'location_new': 'location',
    'last_counted_new': 'last_counted'
}, inplace=True)

In [93]:
# Rename columns in removed_df to remove suffixes
removed_df.rename(columns={
    'name_old': 'name',
    'quantity_old': 'quantity',
    'location_old': 'location',
    'last_counted_old': 'last_counted'
}, inplace=True)

In [95]:
# Rename columns in retained_df to remove suffixes
retained_df.rename(columns={
    'name_old': 'name',
}, inplace=True)

In [99]:
# Changed data type for quantity columns back to integers
# merge with empty columns added nan values and changed them float
new_df['quantity'] = new_df['quantity'].astype('int64')
removed_df['quantity'] = removed_df['quantity'].astype('int64')
retained_df['quantity_old'] = retained_df['quantity_old'].astype('int64')
retained_df['quantity_new'] = retained_df['quantity_new'].astype('int64')
retained_df['inventory_change'] = retained_df['inventory_change'].astype('int64')

In [104]:
# reindex retained_df columns to more logical order
retained_df= retained_df.reindex(columns=[
    'sku',
    'name',
    'location_old',
    'location_new',
    'last_counted_old',
    'last_counted_new',
    'quantity_old',
    'quantity_new',
    'inventory_change'])

In [105]:
retained_df.columns

Index(['sku', 'name', 'location_old', 'location_new', 'last_counted_old',
       'last_counted_new', 'quantity_old', 'quantity_new', 'inventory_change'],
      dtype='str')

Identifies:

Items present in both snapshots (and whether quantities changed)

Items only in snapshot 1 (removed/sold out)

Items only in snapshot 2 (newly added)

Any data quality issues worth flagging


Data issues:

    column names change between files
        name to product_name
        quantity to qty
        location to warehouse
        last_counted to updated_at
    sku
        format (missing dash)
        capital letters
        duplicates
    name
        whitespace
        names matches between skus
    quantity
        decimals
        negative values (handled through duplicated sku merge)
    location
        (none found)
    last_counted
        date format
